In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from nltk import word_tokenize,sent_tokenize
import nltk

<h4>The first crawling attempt</h4>

In [3]:
df=pd.read_csv('character-predictions.csv')
identity=list()
none_index=list()
#Through the name, get the corresponding description of the character
for i in range(df.shape[0]):
    response=requests.get("http://awoiaf.westeros.org/index.php/"+df.iloc[i]['name'].replace(' ','_'))
    results_page=BeautifulSoup(response.content,'lxml')
    try:
        identity.append(results_page.find('div',class_='mw-parser-output').find('p').get_text())
    except:
        identity.append(None)
        none_index.append(i)

In [13]:
identity[363]='NaN'   #Give Nan value to the None

In [14]:
identity_process=list()
#use the word after the first be-verb as the title of the character
for i in range(1946):
    x=nltk.pos_tag(word_tokenize(identity[i]))
    for j in range(len(x)):
        flag=0           #find NN or NNP or not
        a=0              #find is or was or not
        if x[j][0]=='is'or x[j][0]=='was':
            a=1#find is or was 
            for k in range(len(x)-j):
                if x[j+k][1]=='NNP'or x[j+k][1]=='NN':
                    identity_process.append(x[j+k][0])
                    flag=1 #find NNP or NN 
                    break
            if flag==0 and a==1:
                identity_process.append(0)
                break
            if flag==1 and a==1:
                break
    if a==0:
        identity_process.append(0)           
        
len(identity_process)

1946

In [17]:
#invalid title, we don't need it
kinship = ['descendant','son','mate','brother','daughter','wife','sister','child','cousin','uncle','mother','father','husband','wife','spouse','lover']
#if the title is house,we don't need it anymore
house=['Lannister','Tyrell','Arryn','Targaryen','Martell','Baratheon','Tully','Frey','Greyjoy','Stark','Bolton','Brave Companions','Brave','Blackwood',"Night's Watch",'Brave','Blackwood','Night','Ghiscari','Tyroshi']
#get the index of the character whose title is invalid
kinship_index=df[np.array([identity_process[i] in kinship for i in range(1946)])].index
member_index=df[np.array([identity_process[i]=='member' for i in range(1946)])].index
house_index=df[np.array([identity_process[i] in house for i in range(1946)])].index

In [21]:
member_categories=[]
#use the category on the website to subsititute the 'member' title
for i in member_index:
    name=df.iloc[i]['name']
    url="http://awoiaf.westeros.org/index.php/"+name
    response=requests.get(url)
    results_page=BeautifulSoup(response.content,'lxml')
    div=results_page.find('div',class_="mw-normal-catlinks")
    ul=div.find('ul')
    lis=ul.find_all('li')
    member_category=[]
    for li in lis:
        member_category.append(li.get_text())
    member_categories.append((i,member_category))
    
#get the proper title for the 'member'
member_clean=dict()
for idx,i in enumerate(member_index):
    member_clean[i]=list()
    for j in member_categories[idx][1]:# we don't want the following word as title
        if (not('Casualties' in j) and  not('Characters' in j) and not('Dornishmen' in j) and not('Members' in j) and not('House' in j)
            and not('Supporters' in j) and not ('Followers' in j)):
            member_clean[i].append(j)
            
for i in member_clean.keys():
        if len(member_clean[i])==1:
            identity_process[i]=member_clean[i][0]
        else:
            try:
                for j in member_clean[i]:  #unless we don't have title, we will use nobles and noblewomen
                    if j!='Nobles' and j!='Noblewomen':
                        identity_process[i]=j
                        break
            except:
                identity_process[i]=0

In [22]:
categories_kinship=dict()
#use the category on the website to subsititute the invalid kinship
for i in kinship_index:
    name=df.iloc[i]['name'].replace(' ','_')
#     print(name)
    url="http://awoiaf.westeros.org/index.php/"+name
    response=requests.get(url)
    results_page=BeautifulSoup(response.content,'lxml')
    div=results_page.find('div',class_="mw-normal-catlinks")
    ul=div.find('ul')
    lis=ul.find_all('li')
#     print(lis)
    category_kinship=[]
    for li in lis:     # we don't want the following word as title
        if 'House' not in str(li) and 'Characters' not in str(li) and 'Members' not in str(li) and 'Casualties' not in str(li) and 'Dornishmen' not in str(li) and ('Supporters' not in str(li)) and ('Followers' not in str(li)):
            category_kinship.append(li.get_text())
    categories_kinship[i] = category_kinship

#unless we don't have title, we will use nobles and noblewomen
for i in categories_kinship.keys():
        if len(categories_kinship[i])==1:
            identity_process[i]=categories_kinship[i][0]
        else:
            try:
                for j in categories_kinship[i]:
                    if j!='Nobles' and j!='Noblewomen':
                        identity_process[i]=j
                        break
            except:
                identity_process[i]=0

In [24]:
#use the category on the website to subsititute the title with house name
categories_house=dict()
for i in house_index:
    name=df.iloc[i]['name'].replace(' ','_')
#     print(name)
    url="http://awoiaf.westeros.org/index.php/"+name
    response=requests.get(url)
    results_page=BeautifulSoup(response.content,'lxml')
    div=results_page.find('div',class_="mw-normal-catlinks")
    ul=div.find('ul')
    lis=ul.find_all('li')
#     print(lis)
    category_house=[]
    for li in lis:     # we don't want the following word as title
        if 'House' not in str(li) and 'Characters' not in str(li) and 'Members' not in str(li) and 'Casualties' not in str(li) and 'Dornishmen' not in str(li) and ('Supporters' not in str(li)) and ('Followers' not in str(li)):
            category_house.append(li.get_text())
    categories_house[i] = category_house

#unless we don't have title, we will use nobles and noblewomen
for i in categories_house.keys():
        if len(categories_house[i])==1:
            identity_process[i]=categories_house[i][0]
        else:
            try:
                for j in categories_house[i]:
                    if j!='Nobles' and j!='Noblewomen':
                        identity_process[i]=j
                        break
            except:
                identity_process[i]=0    

In [25]:
df['identity']=identity_process
del df['S.No']
del df['actual']
del df['plod']
del df['pred']
del df['alive']
del df['title']
df.to_csv('data_being processing.csv')

<h4>The second crawling attempt</h4>

In [27]:
#the title is still quite strange, so we use the second method to improve. 
df=pd.read_csv('data_being processing.csv')
identity=list()
none_index=list()
#this time we directly use the category in the website 
for i in range(df.shape[0]):
    response=requests.get("http://awoiaf.westeros.org/index.php/"+df.iloc[i]['name'].replace(' ','_'))
    results_page=BeautifulSoup(response.content,'lxml')
    try:
        div=results_page.find('div',class_="mw-normal-catlinks")
        ul=div.find('ul')
        lis=ul.find_all('li')
        category=[]
        for li in lis:
            category.append(li.get_text())
        identity.append((i,category))
    except:
        identity.append((i,None))
        none_index.append(i)

In [28]:
identity_processing=[0]*len(identity)
for i in range(len(identity)):
    identity_processing[i]=(i,list())
    try:
        for j in identity[i][1]:   #we don't want the words below to appear in our title
            if (not('Casualties' in j) and  not('Characters' in j) and not('Dornishmen' in j) and not('Members' in j) and not('House' in j)
                and not('Supporters' in j) and not ('Followers' in j) and not ('Blacks' in j) and not('Death' in j) and not ('Greens' in j)
               and not('Exiles') in j):
                identity_processing[i][1].append(j)
    except:
        identity_processing[i][1].append('NaN')
        
#get the unique title that we have now
unique=set()
for i in identity_processing:
    for j in i[1]:
        unique.add(j)
unique_identity=pd.Series(np.array(list(unique)))

In [29]:
#we will divide all the title in the next 9 categories
level_kings=['Kings of the Stepstones and the Narrow Sea','Kings of Astapor','Kings of the Trident and Kings of the Rivers and the Hills','Kings of the Isles and the Rivers','Kings of the Isles and the North',
       'Kings in the North','Storm Kings','Kings of the Andals, the Rhoynar, and the First Men','Kings of the Iron Islands','Kings of the Reach','Kings of the Rock',
       'Kings-Beyond-the-Wall','Monarchs']
level_princes=['Princes of Dorne','Princes of Summerhall','Princes of Dragonstone']
level_officials=['Most Devout','Hands of the King','High Septons','Translators','Bankers','Executions for treason','Royal consorts',"Septons of the Night's Watch",'Septons','Executions','Priests of the Drowned God','Novices of the Citadel','Red priests','Torturers and executioners','Spymasters','Great Masters','Good Masters','Masters of ships','Masters-at-arms','Masters of laws','Masters of horse','Masters of whisperers','Masters of coin','Kennelmasters','Wise Masters',"Maesters of the Night's Watch",'Archmaesters','Grand Maesters','Maesters']
level_lords=['Lords of the Marches','Lords of Casterly Rock','Lords of the Eyrie','Lords of Highgarden','Lords Declarant','Lords of Winterfell',
       'Lords Confessor','Lords without lands',"Lords of Griffin's Roost",'Lords of Riverrun','Lords of the Iron Islands','Lords of Blackhaven','Lords of Starfall',
       'Lords of Coldmoat','Lords Commander of the Kingsguard',"Lords of Storm's End",'Lords of Dragonstone','Sealords of Braavos','Lords of the Tides','Lords treasurer',"Lord Commanders of the Night's Watch",
       'Lords of Harrenhal','Lords of Sunspear','Lords of the Hornwood','Lords Paramount of the Trident']
level_knights=['Dragonslayers','First Swords of Braavos','Kinslayers','Sworn shields','Sworn swords','Dragonriders','Protectors of the Realm',"Commanders of the City Watch (King's Landing)",'Swords of the Morning',"Commanders of the Night's Watch",'Lord Protectors','Landed knights','Knights','Hedge knights','Mystery knights','Knights of the Gate',"Knights of Griffin's Roost",'Knights of Ninestars','Turncloaks in the War of the Five Kings','Neutrals in the War of the Five Kings']
level_warriors=['Warrior Women',"Veterans of Greyjoy's Rebellion",'Veterans of the Dance of the Dragons','Veterans of the War of the Ninepenny Kings','Veterans of the First Blackfyre Rebellion','Veterans of the Conquest of Dorne','Veterans of the First Dornish War','Veterans of the Faith Militant uprising',
              "Veterans of Robert's Rebellion","Veterans of Aegon's Conquest",'Pit fighters',"Rebels of Robert's Rebellion","Rebels of the First Blackfyre Rebellion",'Gallant Men','Giants','Captain-generals of the Golden Company',"Rangers of the Night's Watch",'Assassins','Courtiers and followers of Aegon VI Targaryen',"Deserters of the Night's Watch",'Skinchangers','Poachers',"Royalists of Robert's Rebellion",
               'Crew of the Sea Bitch','Crew of the Black Wind','Adventurers and explorers','Rebels','Crew and followers of Euron Greyjoy','Captains of the guards',
               'Khals','Spearwives','Wildling leaders','Camp followers','Guards','First Men figures of the Andal invasion','Possible casualties of the Conflict beyond the Wall',
               'Pirates','Spies',"Recruits of the Night's Watch",'Gate captains','Ship captains']
level_nobles=["Craster's wives",'Mistresses and paramours','Nobles','Noblewomen','Lowborn nobles','Wives of Maegor I Targaryen','Mistresses of Aegon IV Targaryen','Squires']
level_smallfolks=['Smallfolk','Musicians','Sailors','Dancers',"Builders of the Night's Watch",'Tongueless','Stewards','Food tasters','Pages using duplicate arguments in template calls','Pages','Disambiguation pages','Sellsails','Prostitutes',"Tyrion Lannister's household",'Cupbearers','Shipwrights','Merchants','Scribes',
                 "Sailors of the Night's Watch",'Artists','Bastards','Orphans of the Greenblood','Wildlings','Acolytes of the Citadel','Mercenaries','Slavers','Singers',
                 'Wet nurses','Slaves and freedmen','Thieves','Sellswords',"Stewards of the Night's Watch",'Smugglers','Handmaids']
level_magicians=['Warlocks','Prophecy','Necromancers','Maegi']

#several people have different titles, so now we have the priority
#king>prince>lord>official>knight>worrior>noble>magician>smallfolk>none
total_titles=[]
for person in identity_processing:
    flag=0
    titles=person[1]
    for king in level_kings:
        if king in titles:
            total_titles.append('king')
            flag=1
            break
    if flag==1:    #improve the efficiency
        continue
    for prince in level_princes:
        if prince in titles:
            total_titles.append('prince')
            flag=1
            break
    if flag==1:
        continue
    for lord in level_lords:
        if lord in titles:
            total_titles.append('lord')
            flag=1
            break
    if flag==1:
        continue
    for official in level_officials:
        if official in titles:
            total_titles.append('official')
            flag=1
            break
    if flag==1:
        continue
    for knight in level_knights:
        if knight in titles:
            total_titles.append('knight')
            flag=1
            break
    if flag==1:
        continue
    for warrior in level_warriors:
        if warrior in titles:
            total_titles.append('warrior')
            flag=1
            break
    if flag==1:
        continue
    for noble in level_nobles:
        if noble in titles:
            total_titles.append('noble')
            flag=1
            break
    if flag==1:
        continue
    for magician in level_magicians:
        if magician in titles:
            total_titles.append('magician')
            flag=1
            break
    if flag==1:
        continue
    for smallfolk in level_smallfolks:
        if smallfolk in titles:
            total_titles.append('smallfolk')
            flag=1
            break
    if flag==0:
        total_titles.append('none')
        
del df['identity']
df['identity']=total_titles
df.to_csv('People with title1.csv')

In [31]:
#we delete several attributes which have too many null values on hand and get the final non-machine learning dataset
df=pd.read_csv('People with title final.csv')
df.drop('Unnamed: 0',axis=1,inplace=True)
house_not_null=df[~df['house'].isnull()]['house'].unique()
dict_house=dict()
#get the mode of the culture of the non-null house
for i in house_not_null:
    try:
        dict_house[i]=df[df['house']==i]['culture'].mode()[0]
    except:
        dict_house[i]=np.NaN
#use the mode to fill the null value of culture
for i in df[df['culture'].isnull()].index:
    try:
        df.loc[i,'culture']=dict_house[df.loc[i,'house']]
    except:
        df.loc[i,'culture']=df.loc[i,'culture']  
        
culture_not_null=df[~df['culture'].isnull()]['culture'].unique()
dict_culture=dict()
for i in culture_not_null:
    try:
        dict_culture[i]=df[df['culture']==i]['house'].mode()[0]
    except:
        dict_culture[i]=np.NaN
for i in df[df['house'].isnull()].index:
    try:
        df.loc[i,'house']=dict_culture[df.loc[i,'culture']]
    except:
        df.loc[i,'house']=df.loc[i,'house']  
#For people with Targaryen in the name, he must belong to house Targaryen
for index,row in df.loc[df['house'].isnull()].iterrows():
    if 'Targaryen' in row['name'].split():
        df.loc[index,'house']='House Targaryen'
df.to_csv('final_data.csv')